In [6]:
import cv2
import numpy as np
import os
import joblib
import pandas as pd
import time

# --- Configuration ---
image_path = "C:/Users/BCI-Lab/Downloads/teamA_dataset/_out_dataset/good_data/00469872.png" # Example image path
# !!! IMPORTANT: Update these filenames for your regression model and scalers !!!
regression_model_filename = "svr_line_position_model.joblib" # Updated model filename
feature_scaler_filename = "scaler_X.joblib" # Scaler for input features (X)
target_scaler_filename = "scaler_y.joblib" # Scaler for target variable (t)

# --- USER-DEFINED OPTIMIZED PARAMETERS ---
# These parameters are used for feature extraction.
# Ensure they are appropriate for the features your regression model was trained on.
OPTIMIZED_PARAMS = {
    "lower_L": 137,
    "upper_L": 255,
    "lower_A": 134,
    "upper_A": 161,
    "lower_B": 138,
    "upper_B": 165,
    "color_morph_kernel_size": 3,
    "edge_morph_kernel_size": 7,
    "canny_thresh1": 18,
    "canny_thresh2": 66,
    "hough_threshold": 57,      # Min votes for a line
    "hough_min_length": 18,     # Min line length
    "hough_max_gap": 17,        # Max gap to connect segments
    "crop_percent": 55,
    "line_center_tolerance_percent": 10 # Percentage of image width for "center" tolerance (if used in features)
}

# --- 1. Load the Regression Model and Scalers ---
try:
    loaded_model = joblib.load(regression_model_filename)
    loaded_feature_scaler = joblib.load(feature_scaler_filename)
    loaded_target_scaler = joblib.load(target_scaler_filename)
    print("SVM regression model and both scalers (features and target) loaded successfully.")
except FileNotFoundError:
    print(f"Error: Model or scaler file not found. Ensure '{regression_model_filename}', '{feature_scaler_filename}', and '{target_scaler_filename}' are in the correct directory.")
    exit()
except Exception as e:
    print(f"Error loading model or scalers: {e}")
    exit()

# --- Get the feature column order directly from the loaded feature scaler ---
try:
    feature_columns_order = loaded_feature_scaler.feature_names_in_.tolist()
    print("Successfully retrieved feature column order from the loaded feature scaler.")
except AttributeError:
    print("Error: The loaded feature scaler does not have 'feature_names_in_'. This attribute is expected.")
    print("Please ensure the feature scaler was fitted on a Pandas DataFrame with feature names.")
    print("Alternatively, you might need to manually define 'feature_columns_order' if this attribute is not available.")
    exit()
except Exception as e:
    print(f"An unexpected error occurred while retrieving feature names from the feature scaler: {e}")
    exit()


# --- 2. Load the Image ---
image = cv2.imread(image_path)
if image is None:
    print(f"Error: Could not read image at '{image_path}'. Please check the path and file integrity.")
    exit()

print(f"\nProcessing image: {os.path.basename(image_path)}")

# --- 3. Image Processing and Feature Extraction ---

# --- 3.1. Crop from top ---
crop_y = int(image.shape[0] * OPTIMIZED_PARAMS["crop_percent"] / 100)
if crop_y >= image.shape[0]:
    crop_y = max(0, image.shape[0] - 1)

cropped_image = image[crop_y:, :].copy()

current_features = {} # This dictionary will store features for the current image
start_time = time.perf_counter()

if cropped_image.shape[0] == 0 or cropped_image.shape[1] == 0:
    print(f"Warning: Cropped image for {os.path.basename(image_path)} is empty. Defaulting to 'No Line Detected' equivalent features.")
    # Ensure all keys expected by feature_columns_order are present
    current_features = {
        "cx": -1.0, "num_detected_lines": 0.0, "avg_line_length": 0.0, "total_line_length": 0.0,
        "std_line_length": 0.0, "avg_line_angle_deg": 0.0, "std_line_angle_deg": 0.0,
        "line_cx_mean": -1.0, "line_cx_std": 0.0, "line_cy_mean": -1.0, "longest_line_length": 0.0,
        "longest_line_angle_deg": 0.0, "mask_pixel_count": 0.0, "mask_area_ratio": 0.0,
        "mask_centroid_x_norm": 0.5, "mask_centroid_y_norm": 0.5,
        "mask_hu_moment_1": 0.0, "mask_hu_moment_2": 0.0, "mask_hu_moment_3": 0.0,
        "mask_hu_moment_4": 0.0, "mask_hu_moment_5": 0.0, "mask_hu_moment_6": 0.0, "mask_hu_moment_7": 0.0,
        "color_mask_pixel_count": 0.0, "color_mask_area_ratio": 0.0, "is_line_detected_binary": 0.0
    }
else:
    lab_cropped = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2LAB)
    l_channel, a_channel, b_channel = cv2.split(lab_cropped)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    l_eq = clahe.apply(l_channel)
    lab_eq = cv2.merge([l_eq, a_channel, b_channel])
    blurred_lab_eq = cv2.medianBlur(lab_eq, 5)

    lower_orange_lab = np.array([OPTIMIZED_PARAMS["lower_L"], OPTIMIZED_PARAMS["lower_A"], OPTIMIZED_PARAMS["lower_B"]])
    upper_orange_lab = np.array([OPTIMIZED_PARAMS["upper_L"], OPTIMIZED_PARAMS["upper_A"], OPTIMIZED_PARAMS["upper_B"]])
    color_mask = cv2.inRange(blurred_lab_eq, lower_orange_lab, upper_orange_lab)

    color_morph_kernel_size = OPTIMIZED_PARAMS["color_morph_kernel_size"]
    color_morph_kernel = np.ones((color_morph_kernel_size, color_morph_kernel_size), np.uint8)
    color_mask_morphed = cv2.morphologyEx(color_mask, cv2.MORPH_OPEN, color_morph_kernel, iterations=1)
    color_mask_morphed = cv2.dilate(color_mask_morphed, color_morph_kernel, iterations=1)
    color_mask_morphed = cv2.morphologyEx(color_mask_morphed, cv2.MORPH_CLOSE, color_morph_kernel, iterations=1)

    gray_cropped = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
    blurred_gray = cv2.GaussianBlur(gray_cropped, (5, 5), 0)
    edge_mask = cv2.Canny(blurred_gray, OPTIMIZED_PARAMS["canny_thresh1"], OPTIMIZED_PARAMS["canny_thresh2"])

    edge_morph_kernel_size = OPTIMIZED_PARAMS["edge_morph_kernel_size"]
    edge_morph_kernel = np.ones((edge_morph_kernel_size, edge_morph_kernel_size), np.uint8)
    edge_mask_morphed = cv2.morphologyEx(edge_mask, cv2.MORPH_CLOSE, edge_morph_kernel, iterations=1)

    final_mask = cv2.bitwise_and(color_mask_morphed, edge_mask_morphed)

    lines = cv2.HoughLinesP(final_mask, 1, np.pi / 180,
                            OPTIMIZED_PARAMS["hough_threshold"],
                            minLineLength=OPTIMIZED_PARAMS["hough_min_length"],
                            maxLineGap=OPTIMIZED_PARAMS["hough_max_gap"])

    # Initialize features with default "no line" values
    num_detected_lines = 0.0
    avg_line_length = 0.0; total_line_length = 0.0; std_line_length = 0.0
    avg_line_angle_deg = 0.0; std_line_angle_deg = 0.0
    line_cx_mean = -1.0; line_cx_std = 0.0; line_cy_mean = -1.0 # Use -1 as a placeholder for "not applicable" or "not found"
    longest_line_length = 0.0; longest_line_angle_deg = 0.0
    is_line_detected_binary = 0.0 # 0 for no line, 1 for line detected
    cx = -1.0 # Overall center of detected lines, -1 if no lines

    if lines is not None:
        is_line_detected_binary = 1.0
        num_detected_lines = float(len(lines))
        all_line_midpoints_x, all_line_midpoints_y, line_lengths, line_angles_rad = [], [], [], []
        max_length_found, angle_of_longest_line = 0.0, 0.0

        for line_segment in lines:
            x1, y1, x2, y2 = line_segment[0]
            length = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
            line_lengths.append(length)
            if length > max_length_found:
                max_length_found = length
                angle_of_longest_line = np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi
            line_angles_rad.append(np.arctan2(y2 - y1, x2 - x1))
            all_line_midpoints_x.append((x1 + x2) / 2.0)
            all_line_midpoints_y.append((y1 + y2) / 2.0)

        if line_lengths: # Check if list is not empty
            avg_line_length = np.mean(line_lengths)
            total_line_length = np.sum(line_lengths)
            std_line_length = np.std(line_lengths) if len(line_lengths) > 1 else 0.0
            longest_line_length = max_length_found
            longest_line_angle_deg = angle_of_longest_line
        if all_line_midpoints_x: # Check if list is not empty
            cx = np.mean(all_line_midpoints_x) # This is the primary 'cx' feature
            line_cx_mean = cx
            line_cx_std = np.std(all_line_midpoints_x) if len(all_line_midpoints_x) > 1 else 0.0
            line_cy_mean = np.mean(all_line_midpoints_y)
        if line_angles_rad: # Check if list is not empty
            normalized_angles_deg = [angle % 180 for angle in np.degrees(line_angles_rad)]
            avg_line_angle_deg = np.mean(normalized_angles_deg)
            std_line_angle_deg = np.std(normalized_angles_deg) if len(normalized_angles_deg) > 1 else 0.0
    
    # Features from the final combined mask
    mask_pixel_count = float(np.sum(final_mask > 0))
    mask_area_ratio = mask_pixel_count / (final_mask.shape[0] * final_mask.shape[1]) if (final_mask.shape[0] * final_mask.shape[1]) > 0 else 0.0
    
    M = cv2.moments(final_mask)
    mask_centroid_x_norm, mask_centroid_y_norm = 0.5, 0.5 # Default to center
    hu_moments = np.zeros(7) # Initialize Hu Moments to zeros
    if M["m00"] != 0:
        mask_centroid_x = M["m10"] / M["m00"]
        mask_centroid_y = M["m01"] / M["m00"]
        if final_mask.shape[1] > 0: mask_centroid_x_norm = mask_centroid_x / final_mask.shape[1]
        if final_mask.shape[0] > 0: mask_centroid_y_norm = mask_centroid_y / final_mask.shape[0]
        hu_moments_calc = cv2.HuMoments(M)
        if hu_moments_calc is not None: hu_moments = hu_moments_calc.flatten()

    # Features from the color mask (after morphology)
    color_mask_pixel_count = float(np.sum(color_mask_morphed > 0))
    color_mask_area_ratio = color_mask_pixel_count / (color_mask_morphed.shape[0] * color_mask_morphed.shape[1]) if (color_mask_morphed.shape[0] * color_mask_morphed.shape[1]) > 0 else 0.0

    current_features = {
        "cx": cx, "num_detected_lines": num_detected_lines,
        "avg_line_length": avg_line_length, "total_line_length": total_line_length,
        "std_line_length": std_line_length, "avg_line_angle_deg": avg_line_angle_deg,
        "std_line_angle_deg": std_line_angle_deg, "line_cx_mean": line_cx_mean,
        "line_cx_std": line_cx_std, "line_cy_mean": line_cy_mean,
        "longest_line_length": longest_line_length, "longest_line_angle_deg": longest_line_angle_deg,
        "mask_pixel_count": mask_pixel_count, "mask_area_ratio": mask_area_ratio,
        "mask_centroid_x_norm": mask_centroid_x_norm, "mask_centroid_y_norm": mask_centroid_y_norm,
        "mask_hu_moment_1": hu_moments[0] if len(hu_moments) > 0 else 0.0,
        "mask_hu_moment_2": hu_moments[1] if len(hu_moments) > 1 else 0.0,
        "mask_hu_moment_3": hu_moments[2] if len(hu_moments) > 2 else 0.0,
        "mask_hu_moment_4": hu_moments[3] if len(hu_moments) > 3 else 0.0,
        "mask_hu_moment_5": hu_moments[4] if len(hu_moments) > 4 else 0.0,
        "mask_hu_moment_6": hu_moments[5] if len(hu_moments) > 5 else 0.0,
        "mask_hu_moment_7": hu_moments[6] if len(hu_moments) > 6 else 0.0,
        "color_mask_pixel_count": color_mask_pixel_count,
        "color_mask_area_ratio": color_mask_area_ratio,
        "is_line_detected_binary": is_line_detected_binary
    }

# Ensure all features are present in the DataFrame, matching the order from training
input_features_df = pd.DataFrame([current_features])
try:
    # Ensure all expected columns are present, fill with NaN if missing, then reorder
    # This handles cases where some features might not be calculated if no lines are found
    # and ensures the DataFrame structure matches what the scaler expects.
    # However, the current_features dictionary should already have all keys with default values.
    input_features_df_ordered = input_features_df.reindex(columns=feature_columns_order, fill_value=0.0)
    # input_features_df_ordered = input_features_df[feature_columns_order] # Original line, assuming all features always present
except KeyError as e:
    print(f"Error: A feature expected by the scaler was not found after attempting to reindex: {e}.")
    print(f"Extracted features keys: {list(current_features.keys())}")
    print(f"Scaler expected features: {feature_columns_order}")
    print("Please ensure all features in 'feature_columns_order' are being calculated and added to 'current_features'.")
    exit()
except Exception as e:
    print(f"Error reordering feature columns: {e}")
    exit()

# --- 4. Scale the Input Features ---
try:
    input_features_scaled = loaded_feature_scaler.transform(input_features_df_ordered)
except ValueError as e:
    print(f"Error during feature scaling: {e}. Provided shape: {input_features_df_ordered.shape}, Columns: {input_features_df_ordered.columns.tolist()}")
    if hasattr(loaded_feature_scaler, 'n_features_in_'): print(f"Scaler expected features: {loaded_feature_scaler.n_features_in_}")
    exit()
except Exception as e:
    print(f"Unexpected error during feature scaling: {e}")
    exit()

# --- 5. Make the Prediction (Output is in Scaled Target Space) ---
scaled_prediction_value = None # Initialize
original_scale_prediction = None # Initialize

# Check if line was detected by OpenCV and set scaled_prediction_value to 0.0 if not
if current_features.get("is_line_detected_binary", 1.0) == 0.0: # Default to 1.0 (line detected) if key is missing for safety
    print("   Info: No line detected by OpenCV. Setting scaled prediction value to 0.0.")
    scaled_prediction_value = 0.0
    original_scale_prediction = 0.0 # Also set original scale prediction to 0 for consistency
else:
    try:
        scaled_prediction_value = loaded_model.predict(input_features_scaled)[0]
        # Only inverse transform if a line was detected and a prediction was made by the model
        try:
            original_scale_prediction = loaded_target_scaler.inverse_transform(np.array([[scaled_prediction_value]]))[0][0]
        except Exception as e:
            print(f"Error during inverse transformation of prediction: {e}")
            print(f"   Scaled Prediction Value (inverse transform failed): {scaled_prediction_value:.4f}")
    except Exception as e:
        print(f"Error during model prediction: {e}")
        # scaled_prediction_value remains None, and original_scale_prediction remains None

end_time = time.perf_counter()
runtime_ms = (end_time - start_time) * 1000 # Calculate runtime after all processing
print(f"\nPrediction runtime (feature extraction, scaling, prediction, inv. transform, override): {runtime_ms:.2f} ms")

# --- 7. Output the Final Prediction ---
print(f"\nPrediction for '{os.path.basename(image_path)}':")
print(f"   Feature 'is_line_detected_binary': {current_features.get('is_line_detected_binary', 'N/A')}") # Show the detection status
if scaled_prediction_value is not None:
    print(f"   Scaled Prediction Value (cx, from model/override): {scaled_prediction_value:.4f}")
else:
    print("   Scaled Prediction Value: Not available (error during prediction or line not detected logic).")

if original_scale_prediction is not None:
    print(f"   Predicted Value (cx, original scale after inverse transform/override): {original_scale_prediction:.4f}")
else:
    print("   Original Scale Prediction: Not available (error during inverse transformation or line not detected logic).")

SVM regression model and both scalers (features and target) loaded successfully.
Successfully retrieved feature column order from the loaded feature scaler.

Processing image: 00469872.png
   Info: No line detected by OpenCV. Setting scaled prediction value to 0.0.

Prediction runtime (feature extraction, scaling, prediction, inv. transform, override): 7.33 ms

Prediction for '00469872.png':
   Feature 'is_line_detected_binary': 0.0
   Scaled Prediction Value (cx, from model/override): 0.0000
   Predicted Value (cx, original scale after inverse transform/override): 0.0000
